In [ ]:
# 225150207111001_1 MUHAMMAD NADHIF_1
# 225150201111002_2 NALENDRA MARCHELO_2
# 225150200111005_3 NARENDRA ATHA ABHINAYA_3
 # 225150200111003_4 YOSUA SAMUEL EDLYN SINAGA_4

## Impor Library yang Dibutuhkan

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

sns.set_style('whitegrid')

## Memuat dan Memeriksa Data

In [ ]:
# Muat dataset
file_path = 'data/Heart_Disease_Prediction.csv'
df = pd.read_csv(file_path)

df['Heart Disease'] = df['Heart Disease'].map({'Presence': 1, 'Absence': 0})

print("5 Baris Pertama Data (setelah mapping):")
display(df.head())

print("\nInformasi Dataset (setelah mapping):")
df.info()

## Exploratory Data Analysis (EDA)

In [ ]:
TARGET_COLUMN = 'Heart Disease'

print("\nStatistik Deskriptif:")
display(df.describe())

print("\nJumlah Nilai yang Hilang per Kolom:")
print(df.isnull().sum())

plt.figure(figsize=(6, 4))
sns.countplot(x=TARGET_COLUMN, data=df)
plt.title('Distribusi Kelas Penyakit Jantung')
plt.xlabel('Status Penyakit Jantung (0 = Tidak, 1 = Ya)')
plt.ylabel('Jumlah Pasien')
plt.show()

plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Heatmap Korelasi Antar Fitur')
plt.show()

## Preprocessing dan Persiapan Data

In [ ]:
# ## Sel Preprocessing dan Persiapan Data (Versi Robust Final)

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import numpy as np

# Diasumsikan 'df' adalah DataFrame mentah yang sudah dimuat
TARGET_COLUMN = 'Heart Disease'

# --- 1. Pembersihan Data Mentah ---
# Hapus semua baris di mana label targetnya kosong. Ini langkah pertama dan terpenting.
df.dropna(subset=[TARGET_COLUMN], inplace=True)
print(f"1. Baris dengan target kosong dihapus. Baris tersisa: {len(df)}")

# --- 2. Pisahkan Fitur dan Target yang Sudah Bersih ---
X = df.drop(TARGET_COLUMN, axis=1)
y = df[TARGET_COLUMN]
print("2. Fitur (X) dan Target (y) telah dipisahkan.")

# --- 3. Preprocessing pada Fitur (X) ---
# Identifikasi tipe-tipe kolom
numerical_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(include='object').columns.tolist() # (Jika ada di masa depan)

# Isi nilai kosong yang MUNGKIN masih ada di kolom FITUR
imputer_num = SimpleImputer(strategy='median')
X[numerical_cols] = imputer_num.fit_transform(X[numerical_cols])
print("3. Missing values pada fitur telah diisi (jika ada).")

# Standarisasi semua fitur numerik
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_preprocessed = pd.DataFrame(X_scaled, columns=X.columns)
print("4. Semua fitur telah distandarisasi.")

# --- 4. Persiapan Akhir untuk Model ---
# Bagi data yang SUDAH BERSIH DAN DIPROSES menjadi data latih dan uji
X_train, X_test, y_train, y_test = train_test_split(
    X_preprocessed, 
    y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)
print(f"\n✅ Data siap. Bentuk X_train: {X_train.shape}")

## Melatih Model Machine Learning (update integrasi MLFlow)

In [ ]:
# ## Melatih Model Machine Learning (Final dengan Koneksi Server MLflow)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from mlflow.models import infer_signature
import mlflow
import mlflow.sklearn

# --- 1. Hubungkan ke Server MLflow ---
# Baris ini SANGAT PENTING untuk mengirim log ke server Docker Anda
mlflow.set_tracking_uri("http://localhost:5000")

# Atur nama eksperimen
mlflow.set_experiment("Prediksi Penyakit Jantung v2")

# --- 2. Mulai Sesi Pencatatan Eksperimen ---
with mlflow.start_run(run_name="RandomForest_Final_Run"):
    
    # Inisialisasi model terlebih dahulu
    model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
    
    # Log SEMUA parameter model (praktik terbaik)
    mlflow.log_params(model.get_params())
    
    # Latih model dengan data training
    print("Memulai proses training model...")
    model.fit(X_train, y_train)
    print("Model berhasil dilatih!")
    
    # Buat signature dari data latih untuk input model
    y_pred_train = model.predict(X_train)
    signature = infer_signature(X_train, y_pred_train)
    
    # Lakukan prediksi pada data uji untuk evaluasi
    y_pred_test = model.predict(X_test)
    
    # Kumpulkan dan log metrik evaluasi
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred_test),
        "recall": recall_score(y_test, y_pred_test),
        "precision": precision_score(y_test, y_pred_test),
        "f1_score": f1_score(y_test, y_pred_test)
    }
    mlflow.log_metrics(metrics)
    
    # Log model sebagai artefak beserta signature-nya
    mlflow.sklearn.log_model(
        sk_model=model,
        name="model",
        signature=signature
    )
    
    print("\n✅ Eksperimen telah berhasil dicatat ke SERVER MLflow.")

In [19]:
import mlflow
print(mlflow.__version__)

3.1.1


## Evaluasi Model

In [ ]:
y_pred = model.predict(X_test)

# akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi Model: {accuracy:.4f}")

# presisi, recall, f1-score
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred))

# confusion matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Prediksi')
plt.ylabel('Aktual')
plt.title('Confusion Matrix')
plt.show()

##  Menyimpan Model yang Telah Dilatih

In [ ]:
model_dir = 'model'
model_path = os.path.join(model_dir, 'random_forest_heart_disease.joblib')

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

joblib.dump(model, model_path)

print(f"Model telah berhasil disimpan di: {model_path}")

# Membuat Data Sintetis dan Simulasi Drift